In [1]:
import numpy as np
import torch
import os
from scipy import signal, sparse

In [3]:
os.chdir("C:\\Users\\Bridget Leonard\\Desktop\\DNN_fMRI")

## Part 3: Voxelwise Encoding Models

Now that we have our stimuli features from the BridgeTower model, we can use them to predict brain responses. To do this, we will need to load in our fMRI data.
To start we will be using sub1 from Popham et al 2021

### 1 Load fMRI data

In [4]:
s1_movie_test = np.load("bridgetower\\data\\moviedata\\S1\\test.npy")
s1_movie_train = np.load("bridgetower\\data\\moviedata\\S1\\train.npy")

In [5]:
s1_alternateithicatom = np.load("bridgetower\\data\\storydata\\S1\\alternateithicatom.npy")
s1_avatar = np.load("bridgetower\\data\\storydata\\S1\\avatar.npy")
s1_howtodraw = np.load("bridgetower\\data\\storydata\\S1\\howtodraw.npy")
s1_legacy = np.load("bridgetower\\data\\storydata\\S1\\legacy.npy")
s1_life = np.load("bridgetower\\data\\storydata\\S1\\life.npy")
s1_myfirstdaywiththeyankees = np.load("bridgetower\\data\\storydata\\S1\\myfirstdaywiththeyankees.npy")
s1_naked = np.load("bridgetower\\data\\storydata\\S1\\naked.npy")
s1_odetostepfather = np.load("bridgetower\\data\\storydata\\S1\\odetostepfather.npy")
s1_souls = np.load("bridgetower\\data\\storydata\\S1\\souls.npy")
s1_undertheinfluence = np.load("bridgetower\\data\\storydata\\S1\\undertheinfluence.npy")

The data is organized as such:

`moviedata`: Inside this folder, there are subfolders for each participant’s data as well as one folder for the stimuli

Within each partcipant’s folder, there is a train.npy and a test.npy file, which contain the brain data for the training and testing data for the models. The shape of each of these is *[n_TR, z, x, y]*, and the last three dimensions should be flattened in order to match the shape of the vertex to voxel mappings in the mappers folder. Data that is located outside of the cortical mask is indicated by np.nan values.
The shape of the data in the stimuli folder is [n_TR, n_features]. The features dimension includes the motion energy features (N=2139), which are then followed by the semantic features (N=985).

`storydata`: Inside this folder, there are subfolders for each participant’s data as well as one folder for the stimuli

Within each participant’s folder, there is a file for each stimulus which contains the brain data. The shape of each of these is *[n_TR, z, x, y]*, and the last three dimensions should be flattened in order to match the shape of the vertex to voxel mappings in the mappers folder. Data that is located outside of the cortical mask is indicated by np.nan values.
The shape of the data in the stimuli folder is [n_TR, n_features]. The features dimension includes the low level features (number of words (N=1), number of phonemes (N=1), and phoneme counts (N=39)), which are then followed by the semantic features (N=985).

In [6]:
# Assuming your data is in a NumPy array with shape (n_tr, z, x, y)
# Creating a mask where the data is not NaN
mask = ~np.isnan(s1_movie_test)

# Apply the mask and then flatten
# This will keep only the non-NaN values
s1_movie_test_flat = s1_movie_test[mask].reshape(s1_movie_test.shape[0], -1)

mask = ~np.isnan(s1_movie_train)

# Apply the mask and then flatten
# This will keep only the non-NaN values
s1_movie_train_flat = s1_movie_train[mask].reshape(s1_movie_train.shape[0], -1)

### 2 Load in feature vectors

In [7]:
# movie data
test = np.load("bridgetower\\feature_vectors\\movie\\test_data.npy")
train00 = np.load("bridgetower\\feature_vectors\\movie\\train_00_data.npy")
train01 = np.load("bridgetower\\feature_vectors\\movie\\train_01_data.npy")
train02 = np.load("bridgetower\\feature_vectors\\movie\\train_02_data.npy")
train03 = np.load("bridgetower\\feature_vectors\\movie\\train_03_data.npy")
train04 = np.load("bridgetower\\feature_vectors\\movie\\train_04_data.npy")
train05 = np.load("bridgetower\\feature_vectors\\movie\\train_05_data.npy")
train06 = np.load("bridgetower\\feature_vectors\\movie\\train_06_data.npy")
train07 = np.load("bridgetower\\feature_vectors\\movie\\train_07_data.npy")
train08 = np.load("bridgetower\\feature_vectors\\movie\\train_08_data.npy")
train09 = np.load("bridgetower\\feature_vectors\\movie\\train_09_data.npy")
train10 = np.load("bridgetower\\feature_vectors\\movie\\train_10_data.npy")
train11 = np.load("bridgetower\\feature_vectors\\movie\\train_11_data.npy")

In [8]:
# We need to combine all the training vectors so that they correspond to the fmri training dataset
train = np.vstack((train00, train01, train02, train03, train04, train05, train06, train07, train08, train09, train10, train11))
train.shape

(3600, 768)

In [9]:
# story data
alternateithicatom = np.load("bridgetower\\feature_vectors\\story\\alternateithicatom_data.npy")
avatar = np.load("bridgetower\\feature_vectors\\story\\avatar.npy")
howtodraw = np.load("bridgetower\\feature_vectors\\story\\howtodraw.npy")
legacy = np.load("bridgetower\\feature_vectors\\story\\legacy.npy")
life = np.load("bridgetower\\feature_vectors\\story\\life.npy")
myfirstdaywiththeyankees = np.load("bridgetower\\feature_vectors\\story\\myfirstdaywiththeyankees.npy")
naked = np.load("bridgetower\\feature_vectors\\story\\naked.npy")
odetostepfather = np.load("bridgetower\\feature_vectors\\story\\odetostepfather.npy")
souls = np.load("bridgetower\\feature_vectors\\story\\souls.npy")
undertheinfluence = np.load("bridgetower\\feature_vectors\\story\\undertheinfluence.npy")

FileNotFoundError: [Errno 2] No such file or directory: 'bridgetower\\feature_vectors\\story\\avatar.npy'

### 3 Voxelwise Encoding Model for Movie Data
We want to create a voxelwise encoding model that captures the relationship between stimuli features from bridgetower and the fmri responses. 

Since our movie data was create with the same temporal alignment (TR = 2) as our fMRI data, we do not need to do any resampling.

#### 3.1 Finite impulse response model
Understanding the Delays: fMRI responses to stimuli are not instantaneous but follow the hemodynamic response function (HRF), which peaks around 4-6 seconds after the stimulus. By selecting delays of 2, 4, 6, and 8 seconds, you're aiming to capture the rise, peak, and fall of this hemodynamic response relative to your features.

In [10]:
# Assuming features is your (3600, 768) feature data
# And TR (time between scans) is 2 seconds
delays = [2, 4, 6, 8]  # Delays in seconds
shifted_features_list = []

for delay in delays:
    shift_amount = delay // 2  # Assuming TR is 2 seconds
    shifted = np.roll(train, shift_amount, axis=0)
    # Optionally, handle edge effects here (e.g., zero-padding or trimming)
    shifted_features_list.append(shifted)

# Stack the shifted arrays to create a 3D array
shifted_features_3d = np.stack(shifted_features_list, axis=-1)

#### 3.2 Linear model

In [11]:
# We will be using L2-regularized linear regression
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import KFold

For each of the 81111 vertices, we want to calculate it's relationship with the 4*768 delayed features. Where k=768 and m is our vertex of interest:

`m = 4k * beta`

So, after running regression, we should have a matrix of m # of rows and 4k # of columns, where each cell contains the beta values or linear weights on the 4k delayed features for each voxel.

We are told in the dataset that data that is located outside of the cortical mask is indicated by np.nan values so we should ignore these in our regression

In [13]:
# Assuming 'features_shifted' is your 3D array (time points x features x delays)
# And 'fmri_data' is your 2D fMRI data (time points x voxels)

# Reshape the feature data for regression
n_time_points, n_features, n_delays = shifted_features_3d.shape
features_reshaped = shifted_features_3d.reshape(n_time_points, n_features * n_delays)

In [15]:
features_reshaped.shape

(3600, 3072)

In [17]:
n_voxels = s1_movie_train_flat.shape[1]
alphas = np.logspace(-6, 6, 50) # Range for alphas
kf = KFold(n_splits=5) # Example of 5-fold cross-validation

best_alphas = np.zeros(n_voxels)
coefficients = np.zeros((n_voxels, features_reshaped.shape[1]))

In [23]:
n_voxels/1423

57.0

In [24]:
import warnings
warnings.filterwarnings("ignore")

# Start with first batch (there will be 57)
for voxel in range(1423):
    ridge_cv = RidgeCV(alphas=alphas, cv=kf)
    ridge_cv.fit(features_reshaped, s1_movie_train_flat[:, voxel])
    best_alphas[voxel] = ridge_cv.alpha_
    coefficients[voxel, :] = ridge_cv.coef_

    # Print checkpoints
    if voxel == round(1423*0.1):
        print("10% done")
    elif voxel == round(1423*0.2):
        print("20% done")
    elif voxel == round(1423*0.3):
        print("30% done")
    elif voxel == round(1423*0.4):
        print("40% done")
    elif voxel == round(1423*0.5):
        print("50% done")
    elif voxel == round(1423*0.6):
        print("60% done")
    elif voxel == round(1423*0.7):
        print("70% done")
    elif voxel == round(1423*0.8):
        print("80% done")
    elif voxel == round(1423*0.9):
        print("90% done")
print("Complete")

In [18]:
import warnings
warnings.filterwarnings("ignore")

for voxel in range(n_voxels):
    ridge_cv = RidgeCV(alphas=alphas, cv=kf)
    ridge_cv.fit(features_reshaped, s1_movie_train_flat[:, voxel])
    best_alphas[voxel] = ridge_cv.alpha_
    coefficients[voxel, :] = ridge_cv.coef_

    # Print checkpoints
    if voxel == round(n_voxels*0.1):
        print("10% done")
    elif voxel == round(n_voxels*0.2):
        print("20% done")
    elif voxel == round(n_voxels*0.3):
        print("30% done")
    elif voxel == round(n_voxels*0.4):
        print("40% done")
    elif voxel == round(n_voxels*0.5):
        print("50% done")
    elif voxel == round(n_voxels*0.6):
        print("60% done")
    elif voxel == round(n_voxels*0.7):
        print("70% done")
    elif voxel == round(n_voxels*0.8):
        print("80% done")
    elif voxel == round(n_voxels*0.9):
        print("90% done")
print("Complete")

KeyboardInterrupt: 